In [6]:
import json
import os

In [2]:
def make_sentences_with_timestamps(ls_words):
    ls_sentences = []
    curr_sentence = []
    sentence_time_stamps = []
    curr_start_time = 0
    for word in ls_words:
        if word['text'][-1] in ['.', '?', '!'] and len(curr_sentence) > 2:
            curr_sentence.append(word['text'])
            ls_sentences.append(' '.join(curr_sentence))
            curr_sentence = []
            sentence_time_stamps.append((curr_start_time, word['end']))
            curr_start_time = word['end']
        else:
            curr_sentence.append(word['text'])
    if len(curr_sentence) > 0:
        ls_sentences.append(' '.join(curr_sentence))
        sentence_time_stamps.append((curr_start_time, ls_words[-1]['end']))
    return ls_sentences, sentence_time_stamps

In [3]:
def split_response_json(ls_words, chunk_num):
    ls_sentences, sentence_time_stamps = make_sentences_with_timestamps(ls_words)
    total_time = ls_words[-1]['end']
    chunk_time = total_time / chunk_num
    ls_paragraphs = []
    curr_paragraph = []
    paragraph_time_stamps = []
    curr_start_time = 0
    for i, sentence in enumerate(ls_sentences):
        if sentence_time_stamps[i][1] - curr_start_time > chunk_time:
            ls_paragraphs.append(' '.join(curr_paragraph))
            curr_paragraph = []
            paragraph_time_stamps.append((curr_start_time, sentence_time_stamps[i][1]))
            curr_start_time = sentence_time_stamps[i][1]
        else:
            curr_paragraph.append(sentence)
    if len(curr_paragraph) > 0:
        ls_paragraphs.append(' '.join(curr_paragraph))
        paragraph_time_stamps.append((curr_start_time, ls_words[-1]['end']))
    return ls_paragraphs, paragraph_time_stamps

In [4]:
def save_paragraphs(ls_paragraphs, paragraph_time_stamps, save_name):
    result_dict = {}
    for i in range(len(ls_paragraphs)):
        result_dict[i] = {'text': ls_paragraphs[i], 'start_time': paragraph_time_stamps[i][0], 'end_time': paragraph_time_stamps[i][1]}
    with open(f'Chunked_Transcripts/{save_name}', 'w') as f:
        json.dump(result_dict, f)

In [9]:
files = os.listdir('Transcripts')
files_with_json = [file for file in files if 'json' in file]

In [11]:
for file in files_with_json:
    data = json.load(open(f'Transcripts/{file}'))
    ls_paragraphs, paragraph_time_stamps = split_response_json(data['words'], 5)
    save_paragraphs(ls_paragraphs, paragraph_time_stamps, file)